In [1]:
!nvidia-smi

Tue Feb 21 22:30:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 60%   83C    P2   283W / 300W |  35770MiB / 49140MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Device

In [2]:
import torch
from torch import nn

In [3]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
torch.cuda.device_count()

1

In [5]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可⽤的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
        for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 张量与GPU

默认情况下，张量是在CPU上创建的

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

## 存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [9]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.6337, 0.9221, 0.4244],
        [0.3506, 0.7622, 0.3700]], device='cuda:0')

## 复制

以将X传输到第⼆个GPU并在那⾥执⾏操作

In [11]:
Z = X.cuda(0) # Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [12]:
Y + Z

tensor([[1.6337, 1.9221, 1.4244],
        [1.3506, 1.7622, 1.3700]], device='cuda:0')

In [13]:
Z.cuda(0) is Z  #Z.cuda(1) is Z

True

# 神经网络与GPU

In [14]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [15]:
net(X)

tensor([[0.6690],
        [0.6690]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)